In [1]:
import pandas as pd
from sqlalchemy import create_engine
import pyodbc


Connection String

In [2]:
connection_string = 'mssql+pyodbc://YOUSSEFLAWENDY\SQLEXPRESS/CustomerManagementData?driver=ODBC+Driver+17+for+SQL+Server'
engine = create_engine(connection_string)

Function to fetch data from a table with error handling

In [3]:
def fetch_data(query, table_name):
    print(f"Fetching data from {table_name}...")
    try:
        data = pd.read_sql(query, engine)
        print(f"Data fetched successfully from {table_name}.")
        return data
    except Exception as e:
        print(f"An error occurred while fetching data from {table_name}: {e}")

SQL queries for each table

In [4]:
queries = {
    'Customers':"""
        SELECT *
        FROM [CustomerManagementData].[dbo].[Customers]
    """, 
    'OrderItems':"""
        SELECT *
        FROM [CustomerManagementData].[dbo].[OrderItems]
    """,
    'OrderPayments':"""
        SELECT *
        FROM [CustomerManagementData].[dbo].[OrderPayments]
    """,
    'OrderReviews':"""
        SELECT *
        FROM [CustomerManagementData].[dbo].[OrderReviews]
    """,
    'Orders':"""
        SELECT *
        FROM [CustomerManagementData].[dbo].[Orders]
    """,
    'ProductsData':"""
        SELECT *
        FROM [CustomerManagementData].[dbo].[ProductsData]
    """,
    'Sellers':"""
        SELECT *
        FROM [CustomerManagementData].[dbo].[Sellers]
    """
}



Fetching data for all tables

In [5]:
CustomersTable_df = fetch_data(queries['Customers'], 'Customers')
OrderItemsTable_df = fetch_data(queries['OrderItems'], 'OrderItems')
OrderPaymentsTable_df = fetch_data(queries['OrderPayments'], 'OrderPayments')
OrderReviewsTable_df = fetch_data(queries['OrderReviews'], 'OrderReviews')
OrdersTable_df = fetch_data(queries['Orders'], 'Orders')
ProductsDataTable_df = fetch_data(queries['ProductsData'], 'ProductsData')
Sellerstable_df = fetch_data(queries['Sellers'], 'Sellers')


Fetching data from Customers...
Data fetched successfully from Customers.
Fetching data from OrderItems...
Data fetched successfully from OrderItems.
Fetching data from OrderPayments...
Data fetched successfully from OrderPayments.
Fetching data from OrderReviews...
Data fetched successfully from OrderReviews.
Fetching data from Orders...
Data fetched successfully from Orders.
Fetching data from ProductsData...
Data fetched successfully from ProductsData.
Fetching data from Sellers...
Data fetched successfully from Sellers.


Function to clean data

In [6]:
def clean_data(df, table_name):
    print(f"Cleaning data for {table_name}...")
    
    # Remove duplicates
    df = df.drop_duplicates()
    
    # Handle missing values (if any) - Example: removing rows with any NaN
    df = df.dropna()
    
    # Reset index after cleaning
    df = df.reset_index(drop=True)
    
    print(f"Data for {table_name} cleaned. Shape: {df.shape}")
    return df

# Clean the fetched data
CustomersTable_df = clean_data(CustomersTable_df, 'Customers')
OrderItemsTable_df = clean_data(OrderItemsTable_df, 'OrderItems')
OrderPaymentsTable_df = clean_data(OrderPaymentsTable_df, 'OrderPayments')
OrderReviewsTable_df = clean_data(OrderReviewsTable_df, 'OrderReviews')
OrdersTable_df = clean_data(OrdersTable_df, 'Orders')
ProductsDataTable_df = clean_data(ProductsDataTable_df, 'ProductsData')
SellersTable_df = clean_data(Sellerstable_df, 'Sellers')

Cleaning data for Customers...
Data for Customers cleaned. Shape: (99441, 9)
Cleaning data for OrderItems...
Data for OrderItems cleaned. Shape: (112650, 7)
Cleaning data for OrderPayments...
Data for OrderPayments cleaned. Shape: (103886, 5)
Cleaning data for OrderReviews...
Data for OrderReviews cleaned. Shape: (99224, 5)
Cleaning data for Orders...
Data for Orders cleaned. Shape: (99441, 8)
Cleaning data for ProductsData...
Data for ProductsData cleaned. Shape: (32951, 2)
Cleaning data for Sellers...
Data for Sellers cleaned. Shape: (3095, 4)


Merge data

In [7]:
# Join Orders and Customers on 'customer_id'
if 'customer_id' in OrdersTable_df.columns and 'customer_id' in CustomersTable_df.columns:
    merged_orders_customers = OrdersTable_df.merge(CustomersTable_df, on='customer_id', how='inner')
    print("Orders and Customers merged successfully.")
else:
    print("Error: 'customer_id' column missing in either Orders or Customers table.")

# Join OrderItems with ProductsData on 'product_id'
if 'product_id' in OrderItemsTable_df.columns and 'product_id' in ProductsDataTable_df.columns:
    merged_items_products = OrderItemsTable_df.merge(ProductsDataTable_df, on='product_id', how='inner')
    print("OrderItems and ProductsData merged successfully.")
else:
    print("Error: 'product_id' column missing in either OrderItems or ProductsData table.")

# First, merge OrderItems with Orders, then with Customers
if 'order_id' in OrderItemsTable_df.columns and 'order_id' in OrdersTable_df.columns:
    merged_items_orders = OrderItemsTable_df.merge(OrdersTable_df[['order_id', 'customer_id']], on='order_id', how='inner')
    if 'customer_id' in CustomersTable_df.columns:
        merged_orders_customers = merged_items_orders.merge(CustomersTable_df[['customer_id', 'customer_unique_id']], on='customer_id', how='inner')
        print("OrderItems, Orders, and Customers merged successfully.")
    else:
        print("Error: 'customer_id' column missing in Customers table.")
else:
    print("Error: 'order_id' column missing in either OrderItems or Orders table.")


Orders and Customers merged successfully.
OrderItems and ProductsData merged successfully.
OrderItems, Orders, and Customers merged successfully.


Calculating total revenue


In [8]:
if 'price' in OrderItemsTable_df.columns:
    # Calculate total revenue for each order item
    OrderItemsTable_df['total_revenue'] = OrderItemsTable_df['price']  # Assuming price is the revenue per item
    
    # Ensure that the merged DataFrame exists before trying to group by
    if merged_orders_customers is not None:
        # Merge to get total revenue per customer
        merged_orders_customers = merged_orders_customers.merge(OrderItemsTable_df[['order_id', 'total_revenue']], on='order_id', how='left')
        total_revenue_per_customer = merged_orders_customers.groupby('customer_id')['total_revenue'].sum()
        
        print("Total Revenue Per Customer:")
        print(total_revenue_per_customer)
    else:
        print("Error: merged_orders_customers is None.")
else:
    print("Error: 'price' column missing in OrderItems table.")


Total Revenue Per Customer:
customer_id
00012a2ce6f8dcda20d059ce98491703     89.80
000161a058600d5901f007fab4c27140     54.90
0001fd6190edaaf884bcaf3d49edf079    179.99
0002414f95344307404f0ace7a26f1d5    149.90
000379cdec625522490c315e70c7a9fb     93.00
                                     ...  
fffcb937e9dd47a13f05ecb8290f4d3e     78.00
fffecc9f79fd8c764f843e9951b11341     54.90
fffeda5b6d849fbd39689bb92087f431     47.90
ffff42319e9b2d713724ae527742af25    199.90
ffffa3172527f765de70084a7e53aae8     43.60
Name: total_revenue, Length: 98666, dtype: float64


In [9]:
# Descriptive statistics for Customers data
print("Customers Table Statistics:")
print(CustomersTable_df.describe())

# Descriptive statistics for merged Orders and Customers
print("Merged Orders and Customers Table Statistics:")
print(merged_orders_customers.describe())

# Get a count of the number of orders per customer
orders_per_customer = merged_orders_customers.groupby('customer_id').size()
print("Orders per Customer:")
print(orders_per_customer)

Customers Table Statistics:
       customer_zip_code_prefix
count              99441.000000
mean               35137.474583
std                29797.938996
min                 1003.000000
25%                11347.000000
50%                24416.000000
75%                58900.000000
max                99990.000000
Merged Orders and Customers Table Statistics:
       order_item_id          price  freight_value  total_revenue
count  157222.000000  157222.000000  157222.000000  157222.000000
mean        1.647301     109.595975      19.549802     109.595975
std         1.620156     167.971810      15.913533     167.971810
min         1.000000       0.850000       0.000000       0.850000
25%         1.000000      36.490000      12.730000      36.490000
50%         1.000000      69.000000      16.110000      69.000000
75%         2.000000     122.990000      20.980000     122.990000
max        21.000000    6735.000000     409.680000    6735.000000
Orders per Customer:
customer_id
00012a2ce6f

In [10]:
# Saving cleaned data to CSV for further analysis
CustomersTable_df.to_csv('cleaned_customers.csv', index=False)
merged_orders_customers.to_csv('merged_orders_customers.csv', index=False)
total_revenue_per_customer.to_csv('total_revenue_per_customer.csv', header=True)
